In [2]:
!pip install netCDF4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 64.3 MB/s eta 0:00:00


In [4]:
DRIVE_PATH = "/content/drive/MyDrive/data606/"

# Set the location of this script in GDrive
SCRIPT_PATH = DRIVE_PATH + "src/"

# Root Path of the data on the cloud drive
DATA_PATH = DRIVE_PATH + "data/"

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Meteorilogical Source Data**

This data is formatted in netCDF format using SCI scientific units such as degrees Kelvin.  This is highly technical and is collected in  many different types of datasets.  The data aggregated into working datasets such as can be found at Kaggle represents a summarization of the most relevant subsets of this data as what makes data analysis practical without becoming an expert in the field.

In [ ]:
# Sample code to read netCDF data
# This is air pressure recorded at 2meters above surface elevation, using long term mean averaging
import numpy as np
f = netCDF4.Dataset('air.2m.mon.ltm.nc')
print(f.variables.keys()) # get all variable names
air = f.variables['air']
print(type(air))
time = f.variables['time']
print(time)

NameError: ignored

which produces output
```
odict_keys(['lat', 'lon', 'time', 'climatology_bounds', 'air', 'valid_yr_count'])
<class 'netCDF4._netCDF4.Variable'>
float32 air(time, lat, lon)
    long_name: Long Term Mean Monthly Mean of Air Temperature
    valid_range: [150. 400.]
    units: degK
    add_offset: 0.0
    scale_factor: 1.0
    missing_value: -9.96921e+36
    precision: 2
    least_significant_digit: 1
    GRIB_id: 11
    GRIB_name: TMP
    var_desc: Air temperature
    level_desc: 2 m
    statistic: Long Term Mean
    parent_stat: Mean
    dataset: NCEP Reanalysis Derived Products
    actual_range: [199.70786 312.07498]
unlimited dimensions:
current shape = (12, 94, 192)
filling on, default _FillValue of 9.969209968386869e+36 used

<class 'netCDF4._netCDF4.Variable'>
float64 time(time)
    long_name: Time
    delta_t: 0000-01-00 00:00:00
    avg_period: 0030-00-00 00:00:00
    prev_avg_period: 0017-00-00 00:00:00
    standard_name: time
    axis: T
    units: hours since 1800-01-01 00:00:0.0
    climatology: climatology_bounds
    climo_period: 1991/01/01 - 2020/12/31
    actual_range: [-15769752. -15761736.]
    ltm_range: [1674264. 1936512.]
    interpreted_actual_range: 0001/01/01 00:00:00 - 0001/12/01 00:00:00
unlimited dimensions:
current shape = (12,)
filling on, default _FillValue of 9.969209968386869e+36 used
```

In [23]:
import netCDF4
import numpy as np
import pandas as pd

In [17]:
f = netCDF4.Dataset(DATA_PATH + 'air.2m.mon.ltm.nc')

In [18]:
# get all variable names
fields = f.variables.keys()
fields

dict_keys(['lat', 'lon', 'time', 'climatology_bounds', 'air', 'valid_yr_count'])

In [19]:
air = f.variables['air']
air

<class 'netCDF4._netCDF4.Variable'>
float32 air(time, lat, lon)
    long_name: Long Term Mean Monthly Mean of Air Temperature
    valid_range: [150. 400.]
    units: degK
    add_offset: 0.0
    scale_factor: 1.0
    missing_value: -9.96921e+36
    precision: 2
    least_significant_digit: 1
    GRIB_id: 11
    GRIB_name: TMP
    var_desc: Air temperature
    level_desc: 2 m
    statistic: Long Term Mean
    parent_stat: Mean
    dataset: NCEP Reanalysis Derived Products
    actual_range: [199.70786 312.07498]
unlimited dimensions: 
current shape = (12, 94, 192)
filling on, default _FillValue of 9.969209968386869e+36 used

In [20]:
time = f.variables['time']
time

<class 'netCDF4._netCDF4.Variable'>
float64 time(time)
    long_name: Time
    delta_t: 0000-01-00 00:00:00
    avg_period: 0030-00-00 00:00:00
    prev_avg_period: 0017-00-00 00:00:00
    standard_name: time
    axis: T
    units: hours since 1800-01-01 00:00:0.0
    climatology: climatology_bounds
    climo_period: 1991/01/01 - 2020/12/31
    actual_range: [-15769752. -15761736.]
    ltm_range: [1674264. 1936512.]
    interpreted_actual_range: 0001/01/01 00:00:00 - 0001/12/01 00:00:00
unlimited dimensions: 
current shape = (12,)
filling on, default _FillValue of 9.969209968386869e+36 used

---

**Load into DataFrame**

---

In [ ]:
df = pd.DataFrame()

In [21]:
# from https://stackoverflow.com/questions/14035148/import-netcdf-file-to-pandas-dataframe
time_arr = netCDF4.num2date(time[:], time.units)

In [15]:
for f in fields:
  print(f)

lat
lon
time
climatology_bounds
air
valid_yr_count


In [25]:
air[:,0]

masked_array(
  data=[[242.17157, 242.15852, 242.14267, ..., 242.2211 , 242.20508,
         242.18855],
        [241.9897 , 241.98251, 241.96916, ..., 242.0237 , 242.01407,
         242.00264],
        [242.08252, 242.08156, 242.07979, ..., 242.11343, 242.10548,
         242.09566],
        ...,
        [253.3357 , 253.33179, 253.33221, ..., 253.33092, 253.33208,
         253.33705],
        [246.64238, 246.63506, 246.6223 , ..., 246.68326, 246.67113,
         246.65752],
        [243.77982, 243.77052, 243.75478, ..., 243.8347 , 243.81627,
         243.79807]],
  mask=False,
  fill_value=1e+20,
  dtype=float32)

In [24]:
ser = pd.Series(air[:,0],index=time_arr)


ValueError: ignored

In [26]:
lat = f.variables['lat']


In [28]:
ser = pd.Series(lat[:],index=time_arr)


ValueError: ignored